In [1]:
pip install pymysql sqlalchemy pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# mysql credentials
username = 'root'
password = 'Devesh123'
host = '127.0.0.1'
port = '3306'
database = 'project_wsdv_2'

csv_path = 'quotes.csv'  


df = pd.read_csv(csv_path)

# Correcting connection string
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# Upload DataFrame to MySQL 
df.to_sql('quotes', con=engine, index=False, if_exists='replace') 

print("Data uploaded successfully!")

# verify upload
df_check = pd.read_sql("SELECT * FROM quotes", con=engine)
print(f"Total records uploaded: {len(df_check)}")
print(df_check.head())


Data uploaded successfully!
Total records uploaded: 100
            Author                                              Quote  \
0  Albert Einstein  “The world as we have created it is a process ...   
1     J.K. Rowling  “It is our choices, Harry, that show what we t...   
2  Albert Einstein  “There are only two ways to live your life. On...   
3      Jane Austen  “The person, be it gentleman or lady, who has ...   
4   Marilyn Monroe  “Imperfection is beauty, madness is genius and...   

                                       Tags  
0       change,deep-thoughts,thinking,world  
1                         abilities,choices  
2  inspirational,life,live,miracle,miracles  
3             aliteracy,books,classic,humor  
4                 be-yourself,inspirational  


In [5]:
# Read the data again from MySQL
df = pd.read_sql("SELECT * FROM quotes", con=engine)
df

,Author,Quote,Tags
0,Albert Einstein,“The world as we have created it is a process ...,"change,deep-thoughts,thinking,world"
1,J.K. Rowling,"“It is our choices, Harry, that show what we t...","abilities,choices"
2,Albert Einstein,“There are only two ways to live your life. On...,"inspirational,life,live,miracle,miracles"
3,Jane Austen,"“The person, be it gentleman or lady, who has ...","aliteracy,books,classic,humor"
4,Marilyn Monroe,"“Imperfection is beauty, madness is genius and...","be-yourself,inspirational"
...,...,...,...
95,Harper Lee,“You never really understand a person until yo...,better-life-empathy
96,Madeleine L'Engle,“You have to write the book that wants to be w...,"books,children,difficult,grown-ups,write,write..."
97,Mark Twain,“Never tell the truth to people who are not wo...,truth
98,Dr. Seuss,"“A person's a person, no matter how small.”",inspirational


#### 1. Number of Quotes by Each Author

In [15]:
query = """
SELECT Author, COUNT(*) AS quote_count
FROM quotes
GROUP BY Author
ORDER BY quote_count DESC;
"""
df_authors = pd.read_sql(query, con=engine)
print("Number of quotes by each author:")
display(df_authors)

Number of quotes by each author:


,Author,quote_count
0,Albert Einstein,10
1,J.K. Rowling,9
2,Marilyn Monroe,7
3,Dr. Seuss,6
4,Mark Twain,6
5,Jane Austen,5
6,C.S. Lewis,5
7,Bob Marley,3
8,Ernest Hemingway,2
9,Charles Bukowski,2


#### 2. Top 5 Most Common Tags

In [16]:
query = """
SELECT tag, COUNT(*) AS tag_count
FROM (
    SELECT TRIM(SUBSTRING_INDEX(SUBSTRING_INDEX(Tags, ',', numbers.n), ',', -1)) AS tag
    FROM quotes
    JOIN (
        SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL 
        SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL 
        SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9 UNION ALL SELECT 10
    ) AS numbers
    ON CHAR_LENGTH(Tags) - CHAR_LENGTH(REPLACE(Tags, ',', '')) >= numbers.n - 1
) AS tags_split
GROUP BY tag
ORDER BY tag_count DESC
LIMIT 5;
"""
df_tags = pd.read_sql(query, con=engine)
print("Top 5 most common tags:")
display(df_tags)


Top 5 most common tags:


,tag,tag_count
0,love,14
1,life,13
2,inspirational,13
3,humor,12
4,books,11


#### 3. Authors with More Than 5 Quotes

In [17]:
query = """
SELECT Author, COUNT(*) AS quote_count
FROM quotes
GROUP BY Author
HAVING COUNT(*) > 5
ORDER BY quote_count DESC;
"""
df_top_authors = pd.read_sql(query, con=engine)
print("Authors with more than 5 quotes:")
display(df_top_authors)


Authors with more than 5 quotes:


,Author,quote_count
0,Albert Einstein,10
1,J.K. Rowling,9
2,Marilyn Monroe,7
3,Dr. Seuss,6
4,Mark Twain,6


#### 4. Longest Quote and Its Author

In [14]:
query = """
SELECT Author, Quote
FROM quotes
ORDER BY CHAR_LENGTH(Quote) DESC
LIMIT 1;
"""
df_longest = pd.read_sql(query, con=engine)
print("Longest quote and its author:")
display(df_longest)

Longest quote and its author:


,Author,Quote
0,Marilyn Monroe,“This life is what you make it. No matter what...
